In [ ]:
!pip3 install scikit-learn==0.23

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from torch.utils.data import DataLoader, Dataset

In [ ]:
X, y = load_breast_cancer(return_X_y=True, as_frame=True)

In [ ]:
X2 = X.copy()
X2['y'] = y
X2.corr()
X

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [ ]:
class CancerDataset(Dataset):

    def __init__(self):
        self.X, self.y = load_breast_cancer(as_frame=True, return_X_y=True)
    
    def __getitem__(self, index):
        X = self.X.iloc[index]
        y = self.y.iloc[index]
        return torch.tensor(X), torch.tensor(y)
    
    def __len__(self):
        return self.X.shape[0]

In [ ]:
ds = CancerDataset()

In [ ]:
ds[398]

(tensor([1.1060e+01, 1.4830e+01, 7.0310e+01, 3.7820e+02, 7.7410e-02, 4.7680e-02,
         2.7120e-02, 7.2460e-03, 1.5350e-01, 6.2140e-02, 1.8550e-01, 6.8810e-01,
         1.2630e+00, 1.2980e+01, 4.2590e-03, 1.4690e-02, 1.9400e-02, 4.1680e-03,
         1.1910e-02, 3.5370e-03, 1.2680e+01, 2.0350e+01, 8.0790e+01, 4.9670e+02,
         1.1200e-01, 1.8790e-01, 2.0790e-01, 5.5560e-02, 2.5900e-01, 9.1580e-02],
        dtype=torch.float64), tensor(1))

In [ ]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.layer_1 = nn.Linear(in_features=30, out_features=15)
        self.layer_2 = nn.Linear(in_features=15, out_features=1)
    
    def forward(self, x):
        x = F.relu(self.layer_1(x))
        x = F.sigmoid(self.layer_2(x))
        return x

In [ ]:
layer_1 = nn.Linear(in_features=4, out_features=2)
layer_1(torch.tensor([1., 2., 3., 4.]))

tensor([-0.0083,  2.2003], grad_fn=<AddBackward0>)

In [ ]:
def train(model, dataset):
    dataloader = DataLoader(dataset, batch_size=128)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=7e-4)
    for epoch in range(1000):
        for b, batch in enumerate(dataloader):

            optimizer.zero_grad()

            X, y = batch
            X = X.cuda().float()
            y = y.cuda().float()

            outputs = model(X)
            loss = criterion(outputs, y)

            loss.backward()
            
            optimizer.step()
            
            print('Epoch {} Batch {} - Loss: {}'.format(epoch, b, loss))

            

In [ ]:
model = Model().cuda()
dataset = CancerDataset()
train(model, dataset)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([57])) that is different to the input size (torch.Size([57, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Streaming output truncated to the last 5000 lines.
Epoch 0 Batch 1 - Loss: 9.305826187133789
Epoch 0 Batch 2 - Loss: 11.43830680847168
Epoch 0 Batch 3 - Loss: 11.352458000183105
Epoch 0 Batch 4 - Loss: 8.584813117980957
Epoch 1 Batch 0 - Loss: 5.427016258239746
Epoch 1 Batch 1 - Loss: 6.592329025268555
Epoch 1 Batch 2 - Loss: 8.800671577453613
Epoch 1 Batch 3 - Loss: 7.9398298263549805
Epoch 1 Batch 4 - Loss: 4.990433216094971
Epoch 2 Batch 0 - Loss: 7.243889331817627
Epoch 2 Batch 1 - Loss: 6.422492504119873
Epoch 2 Batch 2 - Loss: 8.067989349365234
Epoch 2 Batch 3 - Loss: 6.950457572937012
Epoch 2 Batch 4 - Loss: 2.186840057373047
Epoch 3 Batch 0 - Loss: 14.153236389160156
Epoch 3 Batch 1 - Loss: 12.10952091217041
Epoch 3 Batch 2 - Loss: 11.714045524597168
Epoch 3 Batch 3 - Loss: 8.437562942504883
Epoch 3 Batch 4 - Loss: 5.81189489364624
Epoch 4 Batch 0 - Loss: 28.80445671081543
Epoch 4 Batch 1 - Loss: 18.767879486083984
Epoch 4 Batch 2 - Loss: 16.510793685913086
Epoch 4 Batch 3 - Lo

In [ ]:
X, y = dataset[16]
model(X.cuda().float()), y.cuda()

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(tensor([0.6416], device='cuda:0', grad_fn=<SigmoidBackward>),
 tensor(0, device='cuda:0'))